In [7]:
%run ../../common_utils.py

In [2]:
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler
from IPython.utils import io
import unidecode
from catboost import Pool

In [47]:
pip show catboost

Name: catboost
Version: 0.26.1
Summary: Catboost Python Package
Home-page: https://catboost.ai
Author: CatBoost Developers
Author-email: 
License: Apache License, Version 2.0
Location: c:\programdata\anaconda3\lib\site-packages
Requires: graphviz, matplotlib, numpy, pandas, plotly, scipy, six
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [19]:
random_state = 1
number_of_splits = 5

In [37]:
def load_data(val_data=False,path=None):
    print(path)
    train, test, metadata = load_all_data(path=path)
    nonCategorical, categorical = get_cat_and_non_cat_data(metadata)
    all_features = list(train.columns)
    all_features.remove('price')
    numerical_features = ['area_total','area_kitchen','area_living','floor','rooms','ceiling',
        'bathrooms_shared','bathrooms_private','balconies','loggias','phones','building_id','constructed','stories']
#     categorical_to_numerical(train, ['street','address'])
#     categorical_to_numerical(test, ['street','address'])
    if not val_data:
        X_train, y_train, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, test_labels
    else:
        X_train, y_train, X_test, y_test, test_labels = pre_process_numerical(features = all_features, numerical_features = numerical_features, train = train, test = test,
                            outliers_value=7, val_data=val_data, val_split=0.2, random_state=42, scaler="std",
                            add_R="True", add_rel_height="True", droptable=[],
                            one_hot_encode=False, cat_features=categorical, drop_old=True)
        y_train_log = np.log(y_train)
        return X_train, y_train, y_train_log, X_test, y_test, test_labels
X_train, y_train, y_train_log, test_labels = load_data(path='../')

../
Std


In [38]:
selected_features_catboost=['building_id','area_kitchen', 'area_living',
       'rooms', 'ceiling', 'bathrooms_shared', 'bathrooms_private',
       'windows_court', 'windows_street', 'balconies', 'loggias', 'phones',
       'new', 'street', 'address', 'seller', 'layout', 'condition', 'district',
       'constructed', 'stories', 'elevator_without', 'elevator_passenger',
       'material', 'parking', 'heating', 'r', 'rel_height']

X_train = X_train[selected_features_catboost]

In [39]:
X_train['street']=X_train['street'].map(lambda row: unidecode.unidecode(row))
X_train['address']=X_train['address'].map(lambda row: unidecode.unidecode(row))

In [40]:
train, test, metadata = load_all_data(path='../')
nonCategorical, categorical = get_cat_and_non_cat_data(metadata)

categorical.append('street')
categorical.append('address')
X_train = X_train.astype({'street':'string','address':'string','seller':'int','layout':'int','condition':'int','district':'int','material':'int','parking':'int','heating':'int'})
X_train.drop(['area_kitchen', 'area_living', 'rooms', 'ceiling',
       'bathrooms_shared', 'bathrooms_private', 'windows_court', 
       'windows_street', 'balconies', 'loggias', 'phones', 'new', 'constructed',
       'stories', 'elevator_without', 'elevator_passenger',
       'r', 'rel_height'],axis=1,inplace=True)

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23224 entries, 0 to 23284
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   building_id  23224 non-null  float64
 1   street       23224 non-null  string 
 2   address      23224 non-null  string 
 3   seller       23224 non-null  int32  
 4   layout       23224 non-null  int32  
 5   condition    23224 non-null  int32  
 6   district     23224 non-null  int32  
 7   material     23224 non-null  int32  
 8   parking      23224 non-null  int32  
 9   heating      23224 non-null  int32  
dtypes: float64(1), int32(7), string(2)
memory usage: 1.3 MB


In [45]:
def objective(trial,X_train,y_train):
    
    param = {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-1),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    scores = []
        
    cv = GroupKFold(n_splits=number_of_splits)
    groups = X_train["building_id"]
    i = 1
    for train_index, test_index in cv.split(X_train, y_train, groups):
        print("starting on split ",i)
        i+=1
        X_train2, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train2, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
        X_train2.drop(['building_id'], axis=1,inplace=True)
        X_test.drop(['building_id'], axis=1,inplace=True)
#         print("info")
#         X_train2.info()
#         print("cat",categorical_features_indices)
        model = CatBoostRegressor(**param, random_state=random_state,loss_function='RMSE', cat_features=categorical_features_indices)

        pool = Pool(X_train2.values, label=y_train2, cat_features=categorical_features_indices,feature_names=list(X_train2.columns))
        
        model.fit(
            pool,
            eval_set=[(X_test, y_test)],
            verbose=False,
            early_stopping_rounds=100,
        )
        prediction = np.exp(model.predict(X_test))
        score = root_mean_squared_log_error(prediction, np.exp(y_test))
        scores.append(score)
    return np.average(scores)

In [46]:
categorical_features_indices = categorical

study = optuna.create_study(sampler=TPESampler(), direction="minimize")
func = lambda trial: objective(trial, X_train, y_train_log)
study.optimize(func, n_trials=10, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-11-10 19:10:56,575] A new study created in memory with name: no-name-d327d8c3-3bd5-47d1-9c41-bdfcb83e7808
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
[W 2021-11-10 19:10:56,691] Trial 0 failed because of the following error: TypeError('Cannot convert StringArray to numpy.ndarray')
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-46-2f2ece6d5efc>", line 4, in <lambda>
    func = lambda trial: objective(trial, X_train, y_train_log)
  File "<ipython-input-45-598dfaec9d72>", line 38, in objective
    model.fit(
  File "C:\ProgramData\Anaconda3\lib\site-packages\ca

starting on split  1


TypeError: Cannot convert StringArray to numpy.ndarray